In [25]:
from pathlib import Path
import pandas as pd

In [26]:
root_path = Path("experiments")
experiments = [*root_path.glob("*")]

In [27]:
def prepare_csv(metric_path):
    df = pd.read_csv(metric_path.joinpath("results.csv"), index_col=0)
    df["selection_metric"] = metric_path.stem
    return df

In [28]:
exp_dfs = [prepare_csv(e_name) for e_name in experiments]
final_df = pd.concat(exp_dfs)

In [29]:
final_df.to_csv(root_path.joinpath("all_results.csv"))